![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


In [101]:
# Import necessary packages
import pandas as pd
import csv  # For quoting constants
import torch

from transformers import logging
logging.set_verbosity(logging.WARNING)

In [102]:
df = pd.read_csv(
    "data/car_reviews.csv",
    sep = ";"
)

# Display the first few rows to check the data
df.head()

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


# Sentiment Analyzer

In [103]:
from transformers import pipeline, AutoTokenizer
import evaluate 

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

senti = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", tokenizer=tokenizer)



Device set to use cpu


In [104]:
reviews = df['Review']

predicted_labels = []

for review in reviews:
    res = senti(review)
    predicted_labels.append(res[0]['label'])

predictions = [1 if label == 'POSITIVE' else 0 for label in predicted_labels]
print(predictions)

[1, 1, 1, 0, 1]


In [105]:
classes = df['Class']
reference = [1 if cls=="POSITIVE" else 0 for cls in classes]
#print(reference)

accuracy = evaluate.load("accuracy")
f1 = evaluate.load('f1')

result_accuracy = accuracy.compute(predictions=predictions, references=reference)
result_f1 = f1.compute(predictions=predictions, references=reference)

print(f"Accuracy: {result_accuracy['accuracy']}")
print(f"F1: {result_f1['f1']}")

Accuracy: 0.8
F1: 0.8571428571428571


# Translation (English to Espaniol)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
translator = pipeline(task="translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")

Device set to use cpu


In [107]:
sentences = reviews[0].split('.')
#print(sentences)
text = sentences[0] + '.' + sentences[1] + '.'
#print(text)

translated_review = translator(text, clean_up_tokenization_spaces=True)[0]['translation_text']
#print(out[0]['translation_text'])
print(translated_review)

Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.


In [108]:
with open("data/reference_translations.txt", "r") as f:
    references = [f.read().split('\n')]

#print(references)
predictions = [translated_review]
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(predictions=predictions, references=references)
print(bleu_score['bleu'])

0.7794483794144497


# QnA

In [109]:
'''model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
qna = pipeline('question-answering', model=model_name, tokenizer=tokenizer)'''

'model_name = "deepset/roberta-base-squad2"\ntokenizer = AutoTokenizer.from_pretrained(model_name)\nqna = pipeline(\'question-answering\', model=model_name, tokenizer=tokenizer)'

In [110]:
'''QA_input = { 'question' : 'What did he like about the brand?',
             'context' : reviews[1]}

answer = qna(QA_input)
#print(f"Answer: {answer['answer']}")'''

from transformers import AutoModelForQuestionAnswering
# Instantiate model and tokenizer
model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

# Define context and question, and tokenize them
context = reviews[1]
print(f"Context:\n{context}")
question = "What did he like about the brand?"
inputs = tokenizer(question, context, return_tensors="pt")

# Perform inference and extract answer from raw outputs
with torch.no_grad():
  outputs = model(**inputs)
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]

# Decode and show answer
answer = tokenizer.decode(answer_span)
print("Answer: ", answer)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Context:
The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.
Answer:  ride quality, reliability


# Summarization

In [111]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [112]:
last_txt = reviews[len(reviews)-1]
output = summarizer(last_txt, max_length=55, min_length=50)
summarized_text = output[0]['summary_text']
print(summarized_text)

The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. The engine delivers strong


In [113]:
toxicity = evaluate.load('toxicity')
regard = evaluate.load('regard')

Device set to use cpu
Device set to use cpu


In [116]:
#print(summarized_text)

toxicity_result = toxicity.compute(predictions=[summarized_text], aggregation="maximum")
regard_result = regard.compute(data=[summarized_text])

print("Toxicity Result:", toxicity_result)
print("Regard Result:", regard_result)

Toxicity Result: {'max_toxicity': 0.0001512500602984801}
Regard Result: {'regard': [[{'label': 'neutral', 'score': 0.5205848217010498}, {'label': 'negative', 'score': 0.19112174212932587}, {'label': 'other', 'score': 0.1518574059009552}, {'label': 'positive', 'score': 0.1364360898733139}]]}
